In [9]:
import arcpy
from arcpy import env
from arcpy.sa import *
from arcpy.da import *
import math

In [23]:

# 取消并行处理
def disable_parallel_processing(func):
    """"装饰器函数,取消并行处理,因为在部分PC中无法进行并行处理"""
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper

# 坡度(弧度)
@disable_parallel_processing
def get_slope(dem_raser):
    """得到为弧度的坡度"""
    slope_raster = Slope(dem_raser,"DEGREE",1,"PLANAR","METER")
    result_raster = Con(slope_raster==0,0.001, Tan(slope_raster *math.pi/180 ))
    return result_raster


# 填洼
@disable_parallel_processing
def fill_dem(dem_raster):
    """参数默认为最佳"""
    fill_raster = Fill(dem_raster)
    return fill_raster

# 流向
@disable_parallel_processing
def flowdir_dem(dem_raster,dir_index):
    """"D8算法,dir_index{1:正常流动,0:强制外向流动}"""
    if dir_index:
        flowdir_raster = FlowDirection(dem_raster,"NORMAL",None,"D8")
        return flowdir_raster
    else:
        flowdir_raster = FlowDirection(dem_raster,"FORCE", None, "D8")
        return flowdir_raster
    
# 流量(+1)
@disable_parallel_processing
def acc_dem(dem_raster,cell_size):
    """输入流向类型D8"""
    acc_raster = FlowAccumulation(dem_raster,None,"FLOAT","D8")
    result_raster = (acc_raster+1)*cell_size
    return result_raster

# 单位面积汇流量

# 计算TPI
@disable_parallel_processing
def calc_tpi(dem_raster,focous_size):
    """dem_raster:DEM数据,focous_size:窗口大小"""
    # 焦点统计
    focous_raster = FocalStatistics(dem_raster,NbrRectangle(focous_size,focous_size,"CELL"),"MEAN","DATA")
    # 结果输出
    result_raster = dem_raster-focous_raster
    return result_raster


# 计算SCA（单位面积的汇流量）
def calc_sca(flow_acc,flow_dir,cell_size):
    cell_area = cell_size*cell_size
    result_raster = Con(flow_acc==0,cell_area,flow_acc)*cell_area/Con((flow_dir==1)|(flow_dir==8)|(flow_dir==32)|(flow_dir==64),cell_size,cell_area)
    return result_raster

# 计算TWI
@disable_parallel_processing
def calc_twi(dem_raster,cell_size):
    """dem_raster:DEM数据"""
    # 得到正切坡度
    asp_raster = get_slope(dem_raster)
    print("asp done!")
    # 流向
    fd_raster = flowdir_dem(dem_raster,1)
    print("fd done!")
    # sca流量
    ac_raster = acc_dem(fd_raster,cell_size)
    print("acc done!")
    # 计算TWI  
    result_raster = Con(Ln(ac_raster/asp_raster))
    return result_raster


In [24]:
# 设置工作环境
env.workspace = "D:\ArcGISProjects\syraster\SY_TWI.gdb"
dem_file = "SY_DEM_5_DT_3"


twi_raster = calc_twi(dem_file,5)
twi_raster.save("SY_TWI_5")


asp done!
fd done!
acc done!


TypeError: must be real number, not Raster